# Implementing First Order Advection

We now have all the pieces needed to implement first-order accurate advection.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

For convenience, the grid module we already saw can by imported

In [3]:
import fv_grid

Now we'll write our driver.  We'll take as input:

* `nx` : the number of zones
* `u` : the advective velocity (we'll assume $u > 0$)
* `C` : the CFL number

we can also specify the number of periods and pass in the function that supplies the initial conditions

In [ ]:
def advection(nx, u, C, num_periods=1, init_cond=None):
    
    # create the grid
    g = fv_grid.FVGrid(nx, ng=1)
    
    # compute the stop time
    t_period = (g.xmax - g.xmin) / np.abs(u)
    tmax = num_periods * t_period
    
    # setup the initial conditions and store them for later
    # plotting
    init_cond(g)
    g.ainit[:] = g.a[:]
    
    # compute the timestep
    dt = C * g.dx / np.abs(u)
    
    # evolution loop
    t = 0.0
    while t < tmax:
        if t + dt > tmax:
            dt = tmax - t
            
        # get the interface states -- just piecewise constant
        
        # solve the Riemann problem
        
        # compute the fluxes
        
        # conservative update